# Video: Building an Image Classification Model Using LeNet

In [11]:
# 📦 Cell 1: OS and version checks

In [6]:
import torchvision
import torchvision.transforms as transforms

print(torchvision.__version__)

0.21.0


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [15]:
import torchvision.transforms as transforms

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

In [12]:
#SPLIT THE LIBRARIES TO THEIR RESPECTIVE CELLS
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
print(torchvision.__version__)


import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

0.21.0


In [8]:
class HarDataset(Dataset):
    def __init__(self, root_dir, data, transform=None):
        self.root_dir = root_dir
        self.data_df = data
        self.transform = transform
        self.label_map = self._create_label_map()

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.data_df.iloc[idx, 1]
        label = self.label_map[label]

        if self.transform:
            image = self.transform(image)

        return image, label

    def _create_label_map(self):
        unique_labels = sorted(self.data_df['label'].unique())
        label_map = {label: idx for idx, label in enumerate(unique_labels)}
        return label_map

In [17]:
# Define transforms for data augmentation

# Use this for colored images
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Use this for BnW images
# transform = transforms.Compose([
#     transforms.Resize((32, 32)),
#     transforms.Grayscale(num_output_channels=1),  # Convert to black and white
#     transforms.ToTensor(),
# ])

In [18]:
# train_root_dir = r'C:\Users\Deepak\Downloads\Computer Vision Resources\Human Action Recognition\New_train'
# train_file = r'C:\Users\Deepak\Downloads\Computer Vision Resources\Human Action Recognition\Modified_Training_set.csv'
# test_root_dir = r'C:\Users\Deepak\Downloads\Computer Vision Resources\Human Action Recognition\New_test'
# test_file = r'C:\Users\Deepak\Downloads\Computer Vision Resources\Human Action Recognition\Reduced_Testing_set.csv'

In [20]:
train_root_dir = '/Users/pvishnoi/PycharmProjects/data-etl-ml/ml/GenAI-Pinnacle-Master/Computer Vision using PyTorch/Module 2/Human Action Recognition/train'
train_file = '/Users/pvishnoi/PycharmProjects/data-etl-ml/ml/GenAI-Pinnacle-Master/Computer Vision using PyTorch/Module 2/Human Action Recognition/Modified_Training_set.csv'
test_root_dir = '/Users/pvishnoi/PycharmProjects/data-etl-ml/ml/GenAI-Pinnacle-Master/Computer Vision using PyTorch/Module 2/Human Action Recognition/test'
test_file = '/Users/pvishnoi/PycharmProjects/data-etl-ml/ml/GenAI-Pinnacle-Master/Computer Vision using PyTorch/Module 2/Human Action Recognition/Reduced_Testing_set.csv'

In [22]:
# df_train = pd.read_csv(train_file)
# df_test = pd.read_csv(test_file)

In [10]:
df_train, df_val = train_test_split(df_train, test_size=0.2, stratify=df_train.label)

NameError: name 'train_test_split' is not defined

In [ ]:
df_train.shape, df_val.shape

In [ ]:
# Create train and test datasets
train_dataset = HarDataset(root_dir=train_root_dir, data=df_train, transform=transform)
val_dataset = HarDataset(root_dir=train_root_dir, data=df_val, transform=transform)

In [11]:
# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

NameError: name 'DataLoader' is not defined

In [12]:
output_label_map = {val: key for (key, val) in train_dataset.label_map.items()}

NameError: name 'train_dataset' is not defined

In [ ]:
for x, y in train_loader:
    print(x.shape)
    print(y)
    break

In [ ]:
# Count the number of unique labels
num_classes = len(train_dataset.label_map)

In [ ]:
num_classes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Checking class distribution
class_distribution = df_train['label'].value_counts()

# Print the class distribution
print(class_distribution)

# Plotting the class distribution
plt.figure(figsize=(10, 6))
class_distribution.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.title('Class Distribution in the Dataset')
plt.show()


### LeNet Architecture

In [13]:
# Define LeNet-5 model
class LeNet5(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x, 1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

NameError: name 'num_classes' is not defined

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
# Initialize LeNet-5 model
model = LeNet5().to(device)

In [14]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'model' is not defined

In [ ]:
# Train the model
num_epochs = 20
best_accuracy = 0.0
epochs_since_best = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total

    # Evaluate the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100. * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Test Loss: {val_loss:.4f}, Test Accuracy: {val_accuracy:.2f}%')

    # Check for best accuracy and stop if not improved after two more epochs
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        epochs_since_best = 0
        print(f'New best accuracy: {best_accuracy:.2f}%')
    else:
        epochs_since_best += 1
        if epochs_since_best > 2:
            print("Stopping early: no improvement after two consecutive epochs.")
            break


In [ ]:
for idx, row in df_test.sample(10).iterrows():
    img_path = os.path.join(test_root_dir, row['filename'])  
    image_true = Image.open(img_path)  # Open the image file

    # Transform the image and add a batch dimension
    image = transform(image_true).unsqueeze(0).to(device)

    # Pass the image through the model without computing gradients
    with torch.no_grad():
        output = model(image)

    # Get the predicted class label by finding the maximum in the output tensor
    predicted_class = torch.argmax(output).item()

    # Plot the image along with the predicted class label
    plt.figure(figsize=(4, 4))
    plt.imshow(image_true)
    plt.title(f'Prediction: {output_label_map[predicted_class]}')  # Display the predicted label
    plt.show()